<a href="https://colab.research.google.com/github/AnastasiiaDm/machine-learning/blob/main/DZ_15_RNN/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras.preprocessing import sequence


In [3]:
# Load IMDB dataset
max_features = 10000  # Number of words to consider as features
maxlen = 500  # Cut texts after this many words (for padding)

(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data(num_words=max_features)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
# Pad sequences to ensure uniform input size
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

print("X train shape: ",X_train.shape)
print(X_train[5])

X train shape:  (25000, 500)
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0   

In [5]:
# Build the SimpleRNN model
simple_RNN_model = keras.Sequential([
    keras.layers.Embedding(input_dim=max_features, output_dim=128, input_length=maxlen),
    keras.layers.SimpleRNN(64, return_sequences=False),
    keras.layers.Dense(1, activation='sigmoid')
])

simple_RNN_model.build(input_shape=(None, maxlen))
print(simple_RNN_model.summary())

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 500, 128)            │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ (None, 64)                  │          12,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,292,417 (4.93 MB)

 Trainable params: 1,292,417 (4.93 MB)

 Non-trainable params: 0 (0.00 B)

None


In [9]:
simple_RNN_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 32
epochs = 5
simple_RNN_model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

test_loss, test_acc = simple_RNN_model.evaluate(X_test, y_test)
print(f'Test accuracy of simple_RNN_model: {test_acc:.4f}')

Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 187s 237ms/step - accuracy: 0.8309 - loss: 0.3835 - val_accuracy: 0.6805 - val_loss: 0.5980
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 201s 236ms/step - accuracy: 0.8291 - loss: 0.3764 - val_accuracy: 0.6173 - val_loss: 0.6391
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 205s 240ms/step - accuracy: 0.7784 - loss: 0.4853 - val_accuracy: 0.6794 - val_loss: 0.7279
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 178s 228ms/step - accuracy: 0.8800 - loss: 0.2844 - val_accuracy: 0.6308 - val_loss: 0.6377
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 206s 233ms/step - accuracy: 0.7635 - loss: 0.4791 - val_accuracy: 0.7093 - val_loss: 0.6175
782/782 ━━━━━━━━━━━━━━━━━━━━ 32s 41ms/step - accuracy: 0.7050 - loss: 0.6323
Test accuracy of simple_RNN_model: 0.7093


In [10]:
# Build the LSTM model
LSTM_model = keras.Sequential([
    keras.layers.Embedding(input_dim=max_features, output_dim=128, input_length=maxlen),
    keras.layers.LSTM(64, return_sequences=False),
    keras.layers.Dense(1, activation='sigmoid')
])

LSTM_model.build(input_shape=(None, maxlen))
print(LSTM_model.summary())

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 500, 128)            │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,329,473 (5.07 MB)

 Trainable params: 1,329,473 (5.07 MB)

 Non-trainable params: 0 (0.00 B)

None


In [11]:
LSTM_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 32
epochs = 5
LSTM_model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

test_loss, test_acc = LSTM_model.evaluate(X_test, y_test)
print(f'Test accuracy of LSTM_model: {test_acc:.4f}')

Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 374s 473ms/step - accuracy: 0.7058 - loss: 0.5429 - val_accuracy: 0.8617 - val_loss: 0.3351
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 375s 465ms/step - accuracy: 0.8978 - loss: 0.2573 - val_accuracy: 0.8179 - val_loss: 0.3990
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 377s 460ms/step - accuracy: 0.9222 - loss: 0.2000 - val_accuracy: 0.8708 - val_loss: 0.3138
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 384s 462ms/step - accuracy: 0.9483 - loss: 0.1460 - val_accuracy: 0.8741 - val_loss: 0.3453
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 384s 464ms/step - accuracy: 0.9676 - loss: 0.0905 - val_accuracy: 0.8671 - val_loss: 0.4526
782/782 ━━━━━━━━━━━━━━━━━━━━ 72s 92ms/step - accuracy: 0.8670 - loss: 0.4606
Test accuracy of LSTM_model: 0.8671


In [12]:
# Build the GRU model
GRU_model = keras.Sequential([
    keras.layers.Embedding(input_dim=max_features, output_dim=128, input_length=maxlen),
    keras.layers.GRU(64, return_sequences=False),
    keras.layers.Dense(1, activation='sigmoid')
])

GRU_model.build(input_shape=(None, maxlen))
print(GRU_model.summary())

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, 500, 128)            │       1,280,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ (None, 64)                  │          37,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,317,313 (5.03 MB)

 Trainable params: 1,317,313 (5.03 MB)

 Non-trainable params: 0 (0.00 B)

None


In [13]:
GRU_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 32
epochs = 5
GRU_model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

test_loss, test_acc = GRU_model.evaluate(X_test, y_test)
print(f'Test accuracy of GRU_model: {test_acc:.4f}')

Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 415s 527ms/step - accuracy: 0.7183 - loss: 0.5267 - val_accuracy: 0.8652 - val_loss: 0.3195
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 408s 522ms/step - accuracy: 0.8980 - loss: 0.2622 - val_accuracy: 0.8770 - val_loss: 0.2972
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 453s 536ms/step - accuracy: 0.9465 - loss: 0.1515 - val_accuracy: 0.8893 - val_loss: 0.3027
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 414s 500ms/step - accuracy: 0.9746 - loss: 0.0789 - val_accuracy: 0.8828 - val_loss: 0.3927
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 462s 526ms/step - accuracy: 0.9870 - loss: 0.0424 - val_accuracy: 0.8768 - val_loss: 0.4411
782/782 ━━━━━━━━━━━━━━━━━━━━ 66s 84ms/step - accuracy: 0.8753 - loss: 0.4470
Test accuracy of GRU_model: 0.8768


| Model       | Test Accuracy | Training Time per Epoch | Total Training Time |  
|------------|--------------|------------------------|---------------------|  
| Simple RNN | 70.50%       | ~3 min                 | ~15 min             |  
| LSTM       | 86.70%       | ~6 min                 | ~30 min             |  
| GRU        | 87.53%       | ~7 min                 | ~35 min             |  

✅ Best Model: GRU achieved the highest test accuracy with reasonable training time.

⚡ Fastest Model: Simple RNN was the quickest to train but had poor performance.

📈 Most Balanced: LSTM was very close to GRU in accuracy but showed more overfitting.